## TensorFlow学习率设置、正则化与滑动平均模型

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 4.4 神经网络的进一步优化

学习率的设置：指数衰减法  

In [ ]:
#tf.train.exponential_decay函数的实现
decayed_learning_rate = \
   learning_rate * dacay_rate ^ (global_step / decay_steps)

decayed_learning_rate：每一轮优化时使用的学习率  
learning_rate：事先设定的初始学习率  
decay_rate：衰减系数  
decay_steps：衰减速度，当staircase为True时代表完整的使用一遍训练数据所需要的迭代轮数  
tf.train.exponential_decay可以通过设置参数staircase选择不同的衰减方式。staircase默认值False,设置为True时，global_step / decay_steps会被转化成整数，学习率会成为一个阶梯函数。 

In [ ]:
#tf.train.exponential_decay函数的用法
global_step = tf.Variable(0)

#通过exponential_decay函数生成学习率
learning_rate = tf.train.exponential_decay(
    学习率0.1,global_step,轮数100,学习率乘的倍数0.96,staircase=True)

#使用指数衰减的学习率。在minimize函数中传入global_step将自动更新
#global_step参数，从而使得学习率也得到相应更新
learning_step = tf.train.GradientDescentOptimizer(learning_rate)\
                  .minimize(...my loss...,global_step=global_step)

损失函数下降的速度和迭代结束之后总损失的大小没有必然的联系，也就是说并不能通过前几轮损失函数下降的速度来比较不同神经网络的效果。

过拟合：模型很好的记忆了每一个训练数据中随机噪声的部分而忘记了要去学习训练数据中通用的趋势。  
常用的正则化：L1正则化和L2正则化，其中L1正则化会让参数变得更稀疏，而L2不会。L2正则化公式可导，优化更简洁。实践中也可以L1和L2同时使用。  
无论哪一种正则化方式，基本思想都是希望通过限制权重的大小，使得模型不能任意拟合训练数据中的随机噪音。

In [ ]:
#定义一个简单的带L2正则化的损失函数
w = tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y = tf.matmul(x,w)

loss = tf.reduce_mean(tf.square(y_-y)) + #均方误差损失函数
       tf.contrib.layers.l2_regularizer(lambda)(w) #L2正则化，lambda参数表示了正则化项的权重

In [1]:
#使用L1正则化和L2正则化的样例
import tensorflow as tf
weights=tf.constant([[1.0,-2.0],[-3.0,4.0]])
with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l1_regularizer(.5)(weights)))
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(weights)))

5.0
7.5


当神经网络中的参数增多之后，可能会导致损失函数loss的定义很长，可读性差而且容易出错。当网络结构变复杂后，定义网络结构的部分和计算损失函数的部分可能不在同一个函数中，这样通过变量这种方式计算损失函数就不方便了。 
此时可以通过TensorFlow提供的集合(collection)。集合可以在一个计算图（tf.Graph）中保存一组实体（比如张量)。


以下代码给出通过集合计算5层神经网络带L2正则化的损失函数的计算方法：

In [ ]:
# 5层神经网络带L2正则化
import tensorflow as tf

#获取一层神经网络边上的权重，并将这个权重的L2正则化损失加入名称为'losses'的集合中
def get_weight(shape,lambda1):
    #生成一个变量
    var=tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    #add_to_collection 函数将这个新生成变量的L2正则化损失项加入集合
    #这个函数第一个参数losses是集合的名字，第二个参数是要加入这个集合的内容
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(lambda1)(var))
    return var

x=tf.placeholder(tf.float32,shape=(None,2))
y_=tf.placeholder(tf.float32,shape=(None,1))
batch_size=8
#定义了每一层网络中节点的个数
layer_dimension=[2,10,5,3,1]
#神经网络的层数
n_layers=len(layer_dimension)

#这个变量维护前向传播时最深层的节点，开始的时候就是输入层
cur_layer=x
# 当前层的节点个数
in_dimension=layer_dimension[0]

#通过一个循环来生成5层全连接的神经网络结构
for i in range(1,n_layers):
    # layer_demension[i]为下一层的节点数
    out_dimension=layer_dimension[i]
    #生成当前层中权重的变量，并将这个变量的L2正则化损失加入计算图上的集合
    weight=get_weight([in_dimension,out_dimension],0.003)
    bias=tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    #使用Relu激活函数
    cur_layer=tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    #进入下一层之前将下一层的节点个数更新为当前节点个数
    in_dimension=layer_dimension[i]

#在定义神经网络前向传播的同时已经将所有的L2正则化损失加入了图上的集合，
#这里只需要计算刻画模型在训练数据上表现的损失函数
mse_loss=tf.reduce_mean(tf.square(y_-cur_layer))

#将均方误差损失函数加入损失集合
tf.add_to_collection('losses',mse_loss)

#get_collection返回一个列表，这个列表是所有这个集合中的元素。在这个样例中，
#这些元素就是损失函数的不同部分，将它们加起来就可以得到最终的损失函数
loss=tf.add_n(tf.get_collection('losses'))

滑动平均模型：在采用随机梯度下降算法训练神经网络时，使模型在测试数据上更健壮。  
TensorFlow中提供了tf.train.ExponentialMovingAverage来实现滑动平均模型。初始化这个函数时，需要提供一个衰减率。衰减率决定了模型更新的速度，衰减率越大模型越稳定，一般设为0.999或者0.9999。

In [4]:
# 滑动平均模型 ExponentialMovingAverage函数的运用
import tensorflow as tf

#定义一个变量用于计算滑动平均，这个变量的初始值为0。注意这里手动指定了变量类型为tf.float32
#因为所有滑动平均的变量都必须为实数型
v1=tf.Variable(0,dtype=tf.float32)
#这里step变量模拟神经网络中迭代的轮数，可以用于动态控制衰减率
step=tf.Variable(0,trainable=False)

#定义一个滑动平均的类，初始化时给定了衰减率（0.99）和控制衰减率的变量step
ema=tf.train.ExponentialMovingAverage(0.99,step)
#定义一个更新变量滑动平均的操作。这里需要给定一个列表，
#每次执行这个操作时，这个列表中的变量都会被更新
maintain_averages_op=ema.apply([v1])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    #通过ema.average(v1)获取滑动平均之后变量的取值。在初始化之后变量v1的值和v1的滑动平均都为0
    print(sess.run([v1,ema.average(v1)]))
    
    #更新变量v1的值为5
    sess.run(tf.assign(v1,5))
    #更新v1的滑动平均值。衰减率为min{0.99，(1+step)/(10+step)=0.1}=0.1
    #所以v1的滑动平均会被更新为 0.1*0+0.9*5=4.5
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)])) #输出[5.0，4.5]
    
    #更新step的值为10000
    sess.run(tf.assign(step,10000))
    #更新v1的值为10
    sess.run(tf.assign(v1,10))
    #更新v1的滑动平均值。衰减率为min(0.99,(1+10000)/(10+10000))=0.99
    #v1的滑动平均会被更新为0.99*4.5+0.01*10=4.555
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    #[10.0, 4.5549998]
    
    #再次更新滑动平均值，得到的新滑动平均值为 0.99*4.555+0.01*10=4.60945
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
